In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./example.pdf"
# it sucks a bit, PDFs handling is hard, e.g. it seems we lose even symbols like euro sign
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

17


In [3]:
print(docs[0].page_content)
print()
print(docs[0].metadata)

INF
0122
DOCUMENTI INFORMATIVI 
Documenti informativi relativi al contatto per la ricezione e trasmissione di ordini, 
nonché esecuzione per conto del Cliente, collocamento e servizi accessori.  
1. Informativa Pre-contrattuale – cliente al dettaglio – ed. ottobre 2021
2. Informativa Privacy, ai sensi dell’art.13, del Regolamento UE n.679/2016
(regolamento europeo in materia di protezione dei dati personali “GDPR”)
3. Allegato Economico (Allegato 1) – costi e commissioni

{'source': './example.pdf', 'page': 0}


So what just happened?

1. The loader reads the PDF at the specified path into memory.
1. It then extracts text data using the pypdf package.
1. Finally, it creates a LangChain Document for each page of the PDF with the page's content and some metadata about where in the document the text came from

Using a text splitter, you'll split your loaded documents into smaller documents that can more easily fit into an LLM's context window, then load them into a vector store. You can then create a retriever from the vector store for use in our RAG chain.

In [30]:
from langchain_ollama import ChatOllama

# it's dumb
# model = ChatOllama(
#     model="llama3.2",
# )

# it's slow, but smarter
# model = ChatOllama(
#   model="llama3.2-vision"
# )

model = ChatOllama(
  model="mistral-nemo"
)

In [36]:
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings

# slower but embeddings seem to be better
local_embeddings = OllamaEmbeddings(model="mxbai-embed-large")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=local_embeddings
)

retriever = vectorstore.as_retriever()

The final RAG chain:

In [37]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?"})

results

{'input': 'Col profilo semplice, a quanto ammontano le commissioni di trading su EXM?',
 'context': [Document(id='38205b46-08ab-4d76-9244-8290b8dfbf6b', metadata={'source': './example.pdf', 'page': 15}, page_content='ALL1\n0524\n!\n!  \nINFORMAZIONI SUGLI ONERI E SUI COSTI  \n!\nDi seguito sono riportate le commissioni e le condizioni \neconomiche in vigore al  2 maggio 2024. Per successive \nmodifiche si rinvia al sito www.directa.it. \n \nCOMMISSIONI DI TRADING SUI DIVERSI MERCATI  \n \nEXM (ex MTA), EGM, MIV, ETFplus, GEM  \nProfili alternativi: \n•!Semplice: 5! per ordine eseguito \n•!Dinamica*: da 8 a 1,5!  \n•!Variabile: 1,9 per mille per ordine eseguito, con un  \nmassimo di 18! e un minimo di 1,5! (il minimo è di 5! per il \nmercato GEM) per ordini fino a 500.000!   \n \nATFund  \n•!Unico profilo disponibile : 1,9 per mille per ordine esegu ito, \ncon un massimo di 200! e un minimo di 5! per ordini fino a \n500.000!   \n  \nSEDEX e EuroTLX certificati \ndepositario Montetitoli 

In [39]:
print(results["context"][0].metadata)

{'source': './example.pdf', 'page': 15}


In [40]:
rag_chain.invoke({"input": "In media quanto mi costa una operazione fuori mercato?"})

{'input': 'In media quanto mi costa una operazione fuori mercato?',
 'context': [Document(id='af578985-ed48-47fc-86d0-f2ea15693f87', metadata={'source': './example.pdf', 'page': 16}, page_content="•! operazioni fuori mercato con controparte interna a directa       \n       50!  \n•! operazioni fuori mercato con controparte esterna a directa:  \n    - trade tra 0-!1 mln                          9 bps, minimo 50! \n    - trade tra !1 mln – !3 mln               7 bps, minimo 50! \n    - trade tra !3 mln – !5 mln               5 bps, minimo 50!               \n•!girata azionaria su titoli cartacei                                 80! \n \nSERVIZI DI ALERT VIA SMS, SMS TELEGRAM, EMAIL \n \nCon i servizi SMS all'eseguito, SMS a login, SMS Alert e e-mail \nAlert tieni il tuo conto sotto controllo. Puoi ricever e notifiche \nper ogni eseguito, per l’accesso al conto e, tramite gli Alert, \nsu soglie di prezzo da te impostate su titoli di tuo interesse. \n•! Directa Alert                        